# Introduction to Dynamic Circuits and QEC with Qiskit

**Qiskit Slack: @Quantom & @Micheal Healy & Ed Chen**

**Github: taalexander & @mbhealy & @ehchen**

Before running this tutorial make sure you are setup with the [Hello IEEE notebook](./hello-ieee.ipynb).

- This tutorial introduces the current dynamic circuit support through Qiskit on IBM Quantum hardware. This will change quickly as we develop this functionality, and we will aim to keep this tutorial up-to-date with the latest features and functionality. 
- We will then run some very simple error-correction in the form of a bit-flip code
- At the end we will complete some exercises, the solutions may be found in the [solution notebook](./ieee-qec-tutorial-2022-solutions.ipynb).

In [1]:
import os
from typing import Any, List, Dict, Union

import numpy as np
import matplotlib.pyplot as plt

from qiskit import IBMQ, QuantumCircuit, QuantumRegister, ClassicalRegister, quantum_info as qi
from qiskit.providers.ibmq import RunnerResult
from qiskit.result import marginal_counts
import qiskit.tools.jupyter

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [2]:
%load_ext autoreload
%autoreload 2

## Running Dynamic Circuits with Qiskit

Fist we need to instantiate our Qiskit runtime service instance and load our backend.

In [3]:
# Note: This can be any hub/group/project that has access to the required device and the Qiskit runtime.
# Verify that ``qasm3`` is present in ``backend.configuration().supported_features``.
hub = "ibm-q-internal"
group = "dev-sys-software"
project = "internal-test"
backend_name = "ibm_peekskill"

In [4]:
from qiskit.circuit import Delay, Parameter
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend(backend_name, instance=f"{hub}/{group}/{project}")

In [5]:
# Address bug in the current Qiskit runtime package
target = backend.target
if "delay" not in target:
    target.add_instruction(
        Delay(Parameter("t")), {(bit,): None for bit in range(target.num_qubits)}
    )

As we will also enable simulation for most of our experiments to prevent congestion in the reserved hardware we load some simulation backends.

In [6]:
from qiskit.providers.aer import AerSimulator, Aer
from qiskit.providers.aer.noise import NoiseModel

backend_noise_model = NoiseModel.from_backend(backend)
backend_sim = AerSimulator(noise_model=backend_noise_model)
ideal_sim = Aer.get_backend('qasm_simulator')

In [13]:
import utils
backend

<IBMBackend('ibm_peekskill')>

## Running basic quantum error correction

*Note*: This tutorial draws heavily from these excellent tutorials [1](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html) [2](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_quantum_2018/intro2qc/10.Quantum%20error%20correction.ipynb).

A few more excellent references for an introduction to quantum error correction [3](https://www2.physics.ox.ac.uk/sites/default/files/ErrorCorrectionSteane06.pdf) [4](https://arxiv.org/abs/1907.11157) [5](https://arxiv.org/abs/0905.2794)

In [14]:
sim = False # Set True to simulate our experiments and False to run in hardware
dynamical_decoupling = True # Set True to enable dynamical decoupling

In [ ]:
def convert_cycles(time_in_seconds, backend):
    cycles = time_in_seconds / (backend.configuration().dt)
    return int(cycles + (16 - (cycles % 16)))

## Introduction to the repetition code

### The basics of error correction

The basic ideas behind error correction are the same for quantum information as for classical information. This allows us to begin by considering a very straightforward example: speaking on the phone. If someone asks you a question to which the answer is 'yes' or 'no', the way you give your response will depend on two factors:

* How important is it that you are understood correctly?
* How good is your connection?

Both of these can be parameterized with probabilities. For the first, we can use $P_a$, the maximum acceptable probability of being misunderstood. If you are being asked to confirm a preference for ice cream flavours, and don't mind too much if you get vanilla rather than chocolate, $P_a$ might be quite high. If you are being asked a question on which someone's life depends, however, $P_a$ will be much lower.

For the second we can use $p$, the probability that your answer is garbled by a bad connection. For simplicity, let's imagine a case where a garbled 'yes' doesn't simply sound like nonsense, but sounds like a 'no'. And similarly a 'no' is transformed into 'yes'. Then $p$ is the probability that you are completely misunderstood.

A good connection or a relatively unimportant question will result in $p<P_a$. In this case it is fine to simply answer in the most direct way possible: you just say 'yes' or 'no'.

If, however, your connection is poor and your answer is important, we will have $p>P_a$. A single 'yes' or 'no' is not enough in this case. The probability of being misunderstood would be too high. Instead we must encode our answer in a more complex structure, allowing the receiver to decode our meaning despite the possibility of the message being disrupted. The simplest method is the one that many would do without thinking: simply repeat the answer many times. For example say 'yes, yes, yes' instead of 'yes' or 'no, no no' instead of 'no'.

If the receiver hears 'yes, yes, yes' in this case, they will of course conclude that the sender meant 'yes'. If they hear 'no, yes, yes', 'yes, no, yes' or 'yes, yes, no', they will probably conclude the same thing, since there is more positivity than negativity in the answer. To be misunderstood in this case, at least two of the replies need to be garbled. The probability for this, $P$, will be less than $p$. When encoded in this way, the message therefore becomes more likely to be understood. The code cell below shows an example of this.

In [17]:
p1 = 0.01
p3 = 3 * p1**2 * (1-p1) + p1**3 # probability of 2 or 3 errors
print('Probability of a single reply being garbled: {}'.format(p1))
print('Probability of a majority of the three replies being garbled: {:.4f}'.format(p3))

Probability of a single reply being garbled: 0.01
Probability of a majority of the three replies being garbled: 0.0003


If $P<P_a$, this technique solves our problem. If not, we can simply add more repetitions. The fact that $P<p$ above comes from the fact that we need at least two replies to be garbled to flip the majority, and so even the most likely possibilities have a probability of $\sim p^2$. For five repetitions we'd need at least three replies to be garbled to flip the majority, which happens with probability $\sim p^3$. The value for $P$ in this case would then be even lower. Indeed, as we increase the number of repetitions, $P$ will decrease exponentially. No matter how bad the connection, or how certain we need to be of our message getting through correctly, we can achieve it by just repeating our answer enough times.

Though this is a simple example, it contains all the aspects of error correction.
* There is some information to be sent or stored: In this case, a 'yes' or 'no'.
* The information is encoded in a larger system to protect it against noise: In this case, by repeating the message.
* The information is finally decoded, mitigating for the effects of noise: In this case, by trusting the majority of the transmitted messages.

This same encoding scheme can also be used for binary, by simply substituting `0` and `1` for 'yes' and 'no'. It can therefore also be easily generalized to qubits by using the states $\left|0\right\rangle$ and $\left|1\right\rangle$. In each case it is known as the *repetition code*. Many other forms of encoding are also possible in both the classical and quantum cases, which outperform the repetition code in many ways. However, its status as the simplest encoding does lend it to certain applications.

## Quantum Error Correction

While repetition conceptually underlies the codes we will implement in this notebook implementing the repetition code naievly will not allow us to store *quantum information*. This is because the very act of measuring our qubits will destroy the encode state. For example consider if we encode the state $|\Psi_0> = \alpha |0> + \beta |1>$ as $|\tilde{\Psi}_0> = \alpha |000> + \beta |111>$ if we measure all of the qubits according to the Born rule we will obtain the outcome states with probability $Pr(|000>) = |\alpha|^2$ and $Pr(|111>) = |\beta|^2$. 


If we then decode our encoded state $|\tilde{\Psi}_1> \rightarrow |\Psi_1>$ we will obtain

$$
  |\Psi_1> =
\begin{cases}
|0> & \propto Pr(|000>) = |\alpha|^2\\
|1> & \propto Pr(|111>) = |\beta|^2
\end{cases}
$$

Importantly this is **not** the state we encoded $|\Psi_0>$. This is because our measurement operation *does not commute with the encode the state*.

At face value this might imply that quantum error correcting codes are not possible. However, it turns out we can exploit additional ancilla qubits and entanglement to measure what are known as *stabilizers* that do not transform our encoded quantum information, while still informing us of some classes of errors that may have occurred. 

### Stabilizer codes

A quantum stabilizer code encodes $k$ logical qubits into $n$ physical qubits. The coding rate is defined as the ration of $k/n$. The *stabilizer* $S$ is an Abelian subgroup of the Pauli group $\Pi^n$. The +1 eigenspace of the stabilizer operators make up the *codespace* of the code. It will have dimension $2^k$ and therefore encode $k$ qubits. 

Stabilizer codes critically focus on correcting a discrete error set with support from the Pauli group $\Pi^n$. Assume the set of possible errors are $ \epsilon \subset \Pi^n$. For example in a bit flip code with three-qubits encoding the quantum state we will have $\{IIX, IXI, XII\}$.

As both $S$ and $\epsilon$ are subsets of the Pauli group $\Pi^n$ if an error $E \in \epsilon$ is applied to a state it will either commute or anticommute with one of the generators of the Stabilizer group $g \in S$. If the error anticommutes with a Stabilizer element it will be both detectable as it will change the sign of the stabilizer measurement.

In general we can measure each element of the generator of the stabilizer (the minimal representation) $\{g_1, \cdots, g_{n-k} | \forall i \in \{1, \cdots, n-k\}, g_i \in S\}$ which will produce a syndrome bitstring $\vec{x}$ where we assign `0` for a generator that commutes (+1 eigenspace) and `1` for a generator that anticommutes (-1 eigenspace).

We now know how to *detect* an error. The next step is to *correct* the error. It turns out that an error is correctable if it commutes/anticommutes with every generator $g$ in $S$ with the commuted remaining in $S$. If the operator is instead outside the Stabilizer $S$ the error will irrecoverably corrupt the encoded state. In general any two errors, $E_0, E_1 \in \epsilon$ are correctable if $E_0^\dagger E_1 \notin Z(S) \lor E_0^\dagger E_1 \notin S $ where $Z(S)$ is the centralizer of S. In more simple terms if the errors take the Stabilizer into a state that does not lie in the codespace, we will not be able to correct and protect our information.

While the above introduction is a mathematical introduction to Stabilizer codes, we believe its better to learn the practicalities of implementing them with some hands on practice.

### Stabilizer Codes in Practice
In general there is a common flow to most experiments with stabilizer codes. While someday we hope to write a *logical* program and have the hardware determine how to encode and correct erors in the program. In todays NISQ era w are just starting to explore the practical implementation of QEC. Therefore we manually encode our logical state in physical qubits with circuits that we write at the physical qubit level. When we do this there is typically a standard flow to such an experiment:

1. Initialize our input physical state we wish to protect.
2. Encode our state in our Stabilizer's codespace.
3. Apply an error-channel. These may be simulated Krauss maps, probabilitiscally applied gates, or simply the passive error-channel in the device of interest.
4. Measure the syndrome by measuring the generators of our Stabilizer.
5. Apply a decoding sequence to our stabilizer and apply the correction sequence for the error that we have observed. This is where dynamic circuit capabilities are important.
6. Loop to 3. if we are running multiple iterations of the Stabilizer sequence.
7. Decode our encoded state to recover the protected state.
8. Measure the final data qubit to observe the state we protected and determine how well the code performed.

## Executing the Bit-flip code on a Backend

Below we will implement one of the simplest forms of a stabilizer code the five-qubit bit-flip code, which will protect our qubit against a single bit-flip error $\epsilon = \{IIX, IXI, XII \}$

Our quantum circuit will use three data qubits and two ancilla qubits.

In [18]:
# Setup a base quantum circuit for our experiments
qreg_data = QuantumRegister(3)
qreg_measure = QuantumRegister(2)
creg_data = ClassicalRegister(3)
creg_syndrome = ClassicalRegister(2)
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]

def build_qc():
    return QuantumCircuit(qreg_data, qreg_measure, creg_data, creg_syndrome)

### Initialize our qubit
To protect a quantum state we must first prepare it!
In general we could prepare the state $|\Psi_0> = |0000> \otimes (\alpha |0> + \beta |1>)$.
In the circuit below we prepare the physical state $|\Psi_0> = |00001>$.

In [ ]:
qc_init = build_qc()

qc_init.x(qreg_data[0])
qc_init.barrier(qreg_data)

qc_init.draw(output="mpl")

### Encode our logical state
To protect our qubit we must encode it in the codespace. For the case of the bit-flip code this
is very similar to the repetition code, where we implement repetition using the entangling
`CX` gate rather than a classically conditioned bit-flip as we would do in the classical case.

The encoding circuit below will map $|\Psi_1> = U_{en}|\Psi_0> = |00> (\alpha |000> + \beta |111>)$

The stabilizer for the bit flip code is $S = {III, IZZ, ZIZ, ZZI}$. Operationally what this means is that
a single bit-flip error applied to the qubits will modify the parity of a stabilizer but leave it within the code space. It is also straightforward to show that any two non-trivial Stabilizer elements can generate the full stabilizer. For example taking the generator set $G_0 = \{g_0, g_1 \} = \{ IZZ, ZIZ \}$

We can see this as $g_0g_1 = IZZ . ZIZ = ZZI$ and $g_0g_0 = IZZ.IZZ = III$ completing our Stabilizer.

This means that we must only measure our two generators $g_0$ and $g_1$ to detect any *correctable* error.

It is easy to see the prepared state is a +1 eigenstate of our stabilizers
$IZZ |\Psi_1> = |\Psi_1>$ 
$ZIZ |\Psi_2> = |\Psi_2>$ 

This is because the Stabilizer measures the *parity* of the two target qubits.


In [20]:
def encode_bit_flip(qc, state, ancillas):
    control = state
    for ancilla in ancillas:
        qc.cx(control, ancilla)
    qc.barrier(state, *ancillas)
    return qc

qc_encode_bit = build_qc()

encode_bit_flip(qc_encode_bit, state_data, ancillas_data)

qc_encode_bit.draw(output="mpl")

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."

However if we consider the action of bit-flip error $X$ which maps $|0> -> |1>$ and $|1> -> |0>$ on any of our qubits we have $\epsilon = \{E_0, E_1, E_2 \} = \{IIX, IXI, XII\}$ 

Evaluating the commutators of our error terms we see:


$$[E_0, g_0] = [IIX, IZZ] = IIX.IZZ - IZZ.IIX \neq 0$$
$$[E_0, g_1] = [IIX, ZIZ] = IIX.ZIZ - IIX.ZIZ \neq 0$$
$$[E_1, g_0] = [IXI, IZZ] = IXI.IZZ - IZZ.IXI \neq 0$$
$$[E_1, g_1] = [IXI, ZIZ] = IXI.ZIZ - IXI.ZIZ = 0$$
$$[E_2, g_0] = [XII, IZZ] = XII.IZZ - IZZ.XII = 0$$
$$[E_2, g_1] = [XII, ZIZ] = XII.ZIZ - XII.ZIZ \neq 0$$

What we see is that if we measure our stabilizers and observe 
$$
Observe
\begin{cases}
<g_0g_1> = 11, \text{Error on qubit 0} \\
<g_0g_1> = 01, \text{Error on qubit 1} \\
<g_0g_1> = 10, \text{Error on qubit 2}
\end{cases}
$$









### Prepare a decoding circuit
To readout our final state we must map it back from the codespace to a single qubit.
For our code this is simply $U_{de} = U_{en}^\dagger$.

In [ ]:
def decode_bit_flip(qc, state, ancillas):
    inv = qc_encode_bit.inverse()
    return qc.compose(inv)

qc_decode_bit = build_qc()

qc_decode_bit = decode_bit_flip(qc_decode_bit, state_data, ancillas_data)

qc_decode_bit.draw(output="mpl")

### A circuit that prepares our encoded state

Below we see how we can combine the state preparation and encoding steps
to encode our state in the bit-flip code.

In [ ]:
qc_encoded_state_bit = qc_init.compose(qc_encode_bit)
qc_encoded_state_bit.draw(output="mpl")

### Measuring the syndome



In [ ]:
def measure_syndrome_bit(qc, qreg_data, qreg_measure, creg_measure):
    def branch_delay():
        if sim:
            qc.barrier(qreg_data)
            qc.delay(block_branch_cycles, qreg_data)
            qc.barrier(qreg_data)
  
    qc.cx(qreg_data[0], qreg_measure[0])
    qc.cx(qreg_data[1], qreg_measure[0])
    qc.cx(qreg_data[0], qreg_measure[1])
    qc.cx(qreg_data[2], qreg_measure[1])
    qc.measure(qreg_measure, creg_measure)
    branch_delay()
    qc.x(qreg_measure[0]).c_if(creg_measure[0], 1)
    qc.x(qreg_measure[1]).c_if(creg_measure[1], 1)
    qc.barrier(*qreg_data, *qreg_measure)
    return qc

qc_syndrome_bit = measure_syndrome_bit(build_qc(), qreg_data, qreg_measure, creg_syndrome)
qc_syndrome_bit.draw(output="mpl")    

In [ ]:
qc_measure_syndrome_bit = qc_encoded_state_bit.compose(qc_syndrome_bit)
qc_measure_syndrome_bit.draw(output="mpl")

In [22]:
def apply_correction_bit(qc, qreg_data, creg_syndrome):
    # If simulating we need to insert a delay to mirror the hardware
    def branch_delay():
        if sim:
            qc.barrier(qreg_data)
            qc.delay(block_branch_cycles, qreg_data)
            qc.barrier(qreg_data)
    
    branch_delay()
    qc.x(qreg_data[0]).c_if(creg_syndrome, 3)
    branch_delay()
    qc.x(qreg_data[1]).c_if(creg_syndrome, 1)
    branch_delay()
    qc.x(qreg_data[2]).c_if(creg_syndrome, 2)
    qc.barrier(qreg_data)
    return qc
    
qc_correction_bit = apply_correction_bit(build_qc(), qreg_data, creg_syndrome)
qc_correction_bit.draw(output="mpl")

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."

In [ ]:
def apply_final_readout(qc, qreg_data, creg_data):
    """Apply inverse mapping so that we always try and measure |1> in the computational basis.
    
    TODO: The above is just a stand in for proper measurement basis measurement
    """
    qc.barrier(qreg_data)
    qc.measure(qreg_data, creg_data)
    return qc

qc_final_measure = apply_final_readout(build_qc(), qreg_data, creg_data)
qc_final_measure.draw(output="mpl")

In [ ]:
bit_code_circuit = qc_measure_syndrome_bit.compose(qc_correction_bit).compose(qc_final_measure)
bit_code_circuit.draw(output="mpl")

In [ ]:
### Running the bit flip code in hardware

shots = 1024

init_num_resets = 3
init_delay = 0.

favourite_qubits = [5,3,8,2,9]

# Approximate duration of the measurement processing / conditional latency
block_branch_duration = 0.5e-6 
block_branch_cycles = convert_cycles(block_branch_duration, backend)

In [ ]:
def build_error_correction_sequence(
    qc_base: QuantumCircuit,
    qc_init: Optional[QuantumCircuit], 
    qc_encode: QuantumCircuit, 
    qc_channels: List[QuantumCircuit],
    qc_syndrome: QuantumCircuit,
    qc_correct: QuantumCircuit,
    qc_decode: Optional[QuantumCircuit] = None,
    qc_final: Optional[QuantumCircuit] = None,
    name=None,
):
    qc = qc_base
    
    if qc_init:
        qc = qc.compose(
            qc_init
        )
    
    qc = qc.compose(
            qc_encode
        )
    if name is not None:
        qc.name = name
    
    if not qc_channels:
        qc_channels = [QuantumCircuit(*qc.qregs)]
        
    for qc_channel in qc_channels:
        qc = qc.compose(
                qc_channel
            ).compose(
                qc_syndrome
            ).compose(
                qc_correct
            )
    if qc_decode:
        qc = qc.compose(qc_decode)
    
    if qc_final:
        qc = qc.compose(qc_final_measure)
    
    return qc


bit_code_circuit = build_error_correction_sequence(
    build_qc(),
    qc_init,
    qc_encode_bit,
    [],
    qc_syndrome_bit,
    qc_correction_bit,
    None,
    qc_final_measure,
)
bit_code_circuit.draw(output="mpl")

### Running on Hardware

In [ ]:
from qiskit.circuit.library import XGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.instruction_durations import InstructionDurations

def execute(circuits, sim=sim, verbose=True, **kwargs):
    if isinstance(circuits, QuantumCircuit):
        circuits = [circuits]

    if sim:
        return backend_sim.run(circuits, shots=shots, **kwargs)
    else:
        import mapomatic as mm
        deflated_circuits = []
        for circuit in circuits:
            # Required because of https://github.ibm.com/IBM-Q-Software/vt-dynamic-circuits/issues/1106
            deflated_circuit = mm.deflate_circuit(circuit)
            deflated_circuit.name = circuit.name
            deflated_circuits.append(deflated_circuit)
    
        job = run_openqasm3(circuits, backend, verbose=False, shots=shots, init_num_resets=init_num_resets, init_delay=init_delay, **kwargs)
        if verbose:
            print(f"Running on qasm3, job id: {job.job_id}")

        return job
    
def calculate_initial_layout(circuit, mapomatic=mapomatic, favourite_qubits=favourite_qubits, blacklist_qubits=blacklist_qubits):
    deflated = mm.deflate_circuit(circuit)
    if mapomatic:
        deflated = mm.deflate_circuit(circuit)
        available_layouts = [layout for layout in mm.matching_layouts(deflated, backend.configuration().coupling_map, strict_direction=False) if layout[:len(favourite_qubits)] == favourite_qubits and not set(layout).intersection(set(blacklist_qubits))]
        scores = mm.evaluate_layouts(deflated, available_layouts, backend)
        return scores[0][0]
    
    return (favourite_qubits + list(set(range(backend.num_qubits)) - set(favourite_qubits)))[:deflated.num_qubits]

def apply_dynamical_decoupling(circuits, backend, initial_layout):
    from qiskit_ibm_provider.transpiler.passes.scheduling import DynamicCircuitScheduleAnalysis, PadDynamicalDecoupling
    dd_sequence = [XGate(), XGate()]
    durations = InstructionDurations.from_backend(backend)
    pm = PassManager(
        [
            DynamicCircuitScheduleAnalysis(durations),
            PadDynamicalDecoupling(durations, dd_sequence, qubits=initial_layout),
        ]
    )
    return pm.run(circuits)

def apply_transpile(circuits, backend, initial_layout, dynamical_decoupling=dynamical_decoupling, **transpile_kwargs):
    transpiled_circuits = transpile(circuits, backend, initial_layout=initial_layout, optimization_level=3, **transpile_kwargs)
    if dynamical_decoupling:
        return apply_dynamical_decoupling(transpiled_circuits, backend, initial_layout)
    return transpiled_circuits

In [23]:
layout_circuit = transpile(bit_code_circuit, backend, optimization_level=3)
print(initial_layout := calculate_initial_layout(layout_circuit, False, favourite_qubits))

NameError: name 'transpile' is not defined

In [ ]:
transpiled_bit_code_circuit = apply_transpile(bit_code_circuit, backend, initial_layout=initial_layout)

In [ ]:
transpiled_bit_code_circuit.draw()

In [ ]:
result = execute(transpiled_bit_code_circuit).result()

data_indices = list(range(len(qreg_data)))
syndrome_indices = list(range(data_indices[-1]+1, len(qreg_data) + len(qreg_measure) ))
                    
marginalized_data_result = marginal_counts(result, data_indices)
marginalized_syndrome_result = marginal_counts(result, syndrome_indices)

print(f'Completed bit code experiment data measurement counts {marginalized_data_result.get_counts(0)}')
print(f'Completed bit code experiment syndrome measurement counts {marginalized_syndrome_result.get_counts(0)}')

In [ ]:
def decode_result(data_counts, syndrome_counts, verbose=True, indent=0):
    shots = sum(data_counts.values())
    success_trials = data_counts.get('000', 0) + data_counts.get('111', 0)
    failed_trials = shots-success_trials
    error_correction_events = shots-syndrome_counts.get('00', 0)
    
    if verbose:
        print(f"{' ' * indent}Bit flip errors were corrected on {error_correction_events}/{shots} trials")
        print(f"{' ' * indent}A final parity error was detected on {failed_trials}/{shots} trials")
        print(f"{' ' * indent}No error was detected on {success_trials}/{shots} trials")
    return error_correction_events, failed_trials

decode_result(marginalized_data_result.get_counts(0), marginalized_syndrome_result.get_counts(0));